In [24]:
!curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar &> /dev/null

In [25]:
!pip install -U antlr4-python3-runtime &> /dev/null

In [26]:
# ↓↓↓ Celda 2
%%writefile TodoCondicional.g4
grammar TodoCondicional;

// ================== Parser rules ==================
raiz : accion* EOF ;

accion
    : 'if' condicion accion                                                   #Condicional
    | 'task' STRING ('priority' INT)? ('due' STRING)? ('tag' STRING)*         #Agregar
    | 'done' STRING                                                           #Completar
    | 'delete' STRING                                                         #Eliminar
    | 'show' vista                                                            #Mostrar
    ;

vista : 'all' | 'pending' | 'done' ;

// --- Condiciones para IF ---
// if count pending > 2 show pending
// if exists "Hacer informe" done "Hacer informe"
// if has "Hacer informe" == done show all
condicion
    : 'count' vista op INT                    #CountCompare
    | 'exists' STRING                         #ExistsTitle
    | 'has' STRING opEq estado                #TitleStateCompare
    ;


estado: 'pending' | 'done' ;

// =================== Lexer rules ==================


INT       : [0-9]+ ;
STRING    : '"' (~["\r\n])* '"' ;

WS        : [ \t\r\n]+ -> skip ;
LINE_COMMENT : '#' ~[\r\n]* -> skip ;

Overwriting TodoCondicional.g4


In [27]:
!java -jar antlr-4.13.2-complete.jar -Dlanguage=Python3 -visitor TodoCondicional.g4

error(56): TodoCondicional.g4:21:20: reference to undefined rule: op
error(56): TodoCondicional.g4:23:19: reference to undefined rule: opEq


In [28]:
# ↓↓↓ Celda 4 (LIMPIA, MISMA LÓGICA/SALIDA)
from dataclasses import dataclass
from typing import List, Optional
from antlr4 import InputStream, CommonTokenStream
from TodoCondicionalLexer import TodoCondicionalLexer
from TodoCondicionalParser import TodoCondicionalParser
from TodoCondicionalVisitor import TodoCondicionalVisitor


# -------- Modelo de datos --------
@dataclass
class Task:
    """Representa una tarea del lenguaje To-Do."""
    title: str
    done: bool = False
    priority: int = 3
    due: Optional[str] = None
    tags: Optional[List[str]] = None

    def __post_init__(self) -> None:
        # Asegura que tags sea siempre una lista
        if self.tags is None:
            self.tags = []


class TaskList:
    """Estructura que mantiene tareas y un índice por título."""
    def __init__(self) -> None:
        self.tasks: List[Task] = []
        self.index = {}  # título -> posición

    def add(self, title: str, priority: int = 3, due: Optional[str] = None,
            tags: Optional[List[str]] = None) -> bool:
        tags = tags or []
        if title in self.index:
            return False
        self.index[title] = len(self.tasks)
        self.tasks.append(Task(title, False, priority, due, tags))
        return True

    def complete(self, title: str) -> bool:
        i = self.index.get(title)
        if i is None:
            return False
        self.tasks[i].done = True
        return True

    def delete(self, title: str) -> bool:
        i = self.index.get(title)
        if i is None:
            return False
        last = len(self.tasks) - 1
        # Elimina en O(1) intercambiando con el último
        self.tasks[i], self.tasks[last] = self.tasks[last], self.tasks[i]
        self.index[self.tasks[i].title] = i
        self.tasks.pop()
        del self.index[title]
        return True

    def fmt(self, t: Task) -> str:
        """Devuelve la representación de una tarea para impresión."""
        box = "✔" if t.done else " "
        extras: List[str] = []
        if t.priority != 3:
            extras.append(f"P{t.priority}")
        if t.due:
            extras.append(f"due:{t.due}")
        if t.tags:
            extras.append("#" + ",".join(t.tags))
        suf = f"  ({' • '.join(extras)})" if extras else ""
        return f"[{box}] {t.title}{suf}"

    def show_all(self) -> str:
        return "\n".join(self.fmt(t) for t in self.tasks) or "(sin resultados)"

    def show_pending(self) -> str:
        return "\n".join(self.fmt(t) for t in self.tasks if not t.done) or "(sin resultados)"

    def show_done(self) -> str:
        return "\n".join(self.fmt(t) for t in self.tasks if t.done) or "(sin resultados)"


# ---------- Visitor / Ejecuta comandos ----------
class Runner(TodoCondicionalVisitor):
    """Visitor que ejecuta acciones del árbol sintáctico."""
    def __init__(self, tasks: Optional[TaskList] = None) -> None:
        self.tasks = tasks or TaskList()
        self.output_lines: List[str] = []
        self.completed_tasks: List[str] = []  # Títulos de tareas completadas

    # raiz : accion* EOF ;
    def visitRaiz(self, ctx):
        for a in ctx.accion():
            self.visit(a)
        return "\n".join(self.output_lines)

    # ----------- AGREGAR -----------
    def visitAgregar(self, ctx):
        title = self._unquote(ctx.STRING(0).getText())
        priority = 3
        due: Optional[str] = None
        tags: List[str] = []

        # Recorre secuencialmente los opcionales: priority, due, tag*
        n = ctx.getChildCount()
        i = 2  # saltamos "task" y el primer STRING (título)
        while i < n:
            t = ctx.getChild(i).getText()
            if t == "priority" and i + 1 < n:
                priority = int(ctx.getChild(i + 1).getText())
                i += 2
                continue
            if t == "due" and i + 1 < n:
                due = self._unquote(ctx.getChild(i + 1).getText())
                i += 2
                continue
            if t == "tag" and i + 1 < n:
                tags.append(self._unquote(ctx.getChild(i + 1).getText()))
                i += 2
                continue
            i += 1

        ok = self.tasks.add(title, priority, due, tags)
        if not ok:
            self.output_lines.append(f'[!] Ya existe: "{title}"')
        return None

    # ----------- COMPLETAR -----------
    def visitCompletar(self, ctx):
        title = self._unquote(ctx.STRING().getText())
        if self.tasks.complete(title):
            self.completed_tasks.append(title)
        else:
            self.output_lines.append(f'[!] No existe: "{title}"')
        return None

    # ----------- ELIMINAR -----------
    def visitEliminar(self, ctx):
        title = self._unquote(ctx.STRING().getText())
        if not self.tasks.delete(title):
            self.output_lines.append(f'[!] No existe: "{title}"')
        return None

    # ----------- MOSTRAR -----------
    def visitMostrar(self, ctx):
        v = ctx.vista().getText()
        if v == "all":
            out = self.tasks.show_all()
        elif v == "pending":
            out = self.tasks.show_pending()
        else:
            out = self.tasks.show_done()
        self.output_lines.append(out)
        return None

    # ----------- CONDICIONAL -----------
    def visitCondicional(self, ctx):
        cond_ok = self._eval_cond(ctx.condicion())
        condition_text = self._get_condition_text(ctx.condicion())
        self.output_lines.append(f"Condición: {condition_text}")
        if cond_ok:
            self.output_lines.append("Resultado de la condición: Verdadera")
            # Captura la salida de la acción dentro del condicional
            original_output = list(self.output_lines)
            self.output_lines = []
            self.visit(ctx.accion())
            action_output = "\n".join(self.output_lines)
            self.output_lines = original_output
            self.output_lines.append(action_output)
        else:
            self.output_lines.append("Resultado de la condición: Falsa")
        return None

    # Texto legible de la condición (para imprimir)
    def _get_condition_text(self, c) -> str:
        from TodoCondicionalParser import TodoCondicionalParser as P
        if isinstance(c, P.CountCompareContext):
            return f"count {c.vista().getText()} {c.op().getText()} {c.INT().getText()}"
        if isinstance(c, P.ExistsTitleContext):
            return f"exists {c.STRING().getText()}"
        if isinstance(c, P.TitleStateCompareContext):
            return f"has {c.STRING().getText()} {c.opEq().getText()} {c.estado().getText()}"
        return "Condición desconocida"

    # --- Evaluación de condiciones ---
    def _eval_cond(self, c) -> bool:
        # Comprobación por tipo de contexto generado por ANTLR
        from TodoCondicionalParser import TodoCondicionalParser as P

        # count vista op INT
        if isinstance(c, P.CountCompareContext):
            view = c.vista().getText()          # "all" | "pending" | "done"
            op = c.op().getText()               # ">" | "<" | "==" | "!="
            num = int(c.INT().getText())

            count = {
                "all": len(self.tasks.tasks),
                "pending": sum(1 for t in self.tasks.tasks if not t.done),
                "done": sum(1 for t in self.tasks.tasks if t.done),
            }[view]

            if op == ">":
                return count > num
            if op == "<":
                return count < num
            if op == "==":
                return count == num
            if op == "!=":
                return count != num
            return False

        # exists "título"
        if isinstance(c, P.ExistsTitleContext):
            title = self._unquote(c.STRING().getText())
            return title in self.tasks.index

        # has "título" (==|!=) (pending|done)
        if isinstance(c, P.TitleStateCompareContext):
            title = self._unquote(c.STRING().getText())
            op = c.opEq().getText()            # "==" | "!="
            state = c.estado().getText()       # "pending" | "done"

            idx = self.tasks.index.get(title)
            if idx is None:
                return False
            is_done = self.tasks.tasks[idx].done
            want_done = (state == "done")
            return (is_done == want_done) if op == "==" else (is_done != want_done)

        # Caso por defecto
        return False

    @staticmethod
    def _unquote(s: str) -> str:
        """Quita comillas dobles si existen."""
        return s[1:-1] if len(s) >= 2 and s[0] == '"' and s[-1] == '"' else s


# ---------- Función de alto nivel: ejecutar desde string ----------
def run_program(source: str, tasks: Optional[TaskList] = None) -> str:
    """
    Ejecuta un programa del DSL recibido como texto.
    Mantiene el comportamiento original: procesa línea a línea, permitiendo comentarios.
    """
    inp = InputStream(source)
    lexer = TodoCondicionalLexer(inp)
    tokens = CommonTokenStream(lexer)
    parser = TodoCondicionalParser(tokens)
    parser.raiz()  # Se mantiene la llamada (aunque la ejecución real es por línea)

    runner = Runner(tasks)

    # Procesamiento línea a línea (mismo comportamiento que tu versión)
    lines = source.strip().split("\n")
    for line in lines:
        stripped = line.strip()
        if stripped.startswith("#"):
            runner.output_lines.append(stripped)
            runner.output_lines.append("")  # Línea en blanco tras comentario
        elif stripped:
            try:
                line_inp = InputStream(stripped)
                line_lexer = TodoCondicionalLexer(line_inp)
                line_tokens = CommonTokenStream(line_lexer)
                line_parser = TodoCondicionalParser(line_tokens)
                line_tree = line_parser.accion()  # Se asume una acción por línea
                runner.visit(line_tree)
                runner.output_lines.append("")  # Línea en blanco tras la salida
            except Exception as e:
                runner.output_lines.append(f"[!] Error procesando línea: '{stripped}' - {e}")
                runner.output_lines.append("")

    output = "\n".join(runner.output_lines)
    if runner.completed_tasks:
        output += (
            "\n\n--- Tareas completadas ---\n"
            + "\n".join(f"[✔] {t}" for t in runner.completed_tasks)
        )
    return output


# ---------- Utilidad: ejecutar y devolver solo las completadas ----------
def run_and_get_completed(source: str, tasks: Optional[TaskList] = None) -> List[str]:
    """Ejecuta el programa y devuelve la lista de títulos de tareas completadas."""
    inp = InputStream(source)
    lexer = TodoCondicionalLexer(inp)
    tokens = CommonTokenStream(lexer)
    parser = TodoCondicionalParser(tokens)
    tree = parser.raiz()
    runner = Runner(tasks)
    runner.visit(tree)
    return runner.completed_tasks


In [29]:
# ↓↓↓ Celda 5
demo = '''
task "Estudiar compiladores" priority 2 due "2025-12-15" tag "uni"
task "ir al baño xd "
task "Comprar leche" tag "casa"
task "Hacer ejercicio" priority 3
task "Llamar a mamá"
task "Preparar cena" tag "casa"

# Mostrando todas las tareas iniciales
show all

# Marcando tareas como completadas
done "Comprar leche"
done "Llamar a mamá"

# Condición: Si "Estudiar compiladores" existe, mostrar todas las tareas
if exists "Estudiar compiladores" show all

# Eliminando tarea
delete "Hacer ejercicio"

# Completando otra tarea
done "Preparar cena"

# Mostrando todas las tareas finales
show all
'''
print(run_program(demo))







# Mostrando todas las tareas iniciales

[ ] Estudiar compiladores  (P2 • due:2025-12-15 • #uni)
[ ] ir al baño xd 
[ ] Comprar leche  (#casa)
[ ] Hacer ejercicio
[ ] Llamar a mamá
[ ] Preparar cena  (#casa)

# Marcando tareas como completadas



# Condición: Si "Estudiar compiladores" existe, mostrar todas las tareas

Condición: exists "Estudiar compiladores"
Resultado de la condición: Verdadera
[ ] Estudiar compiladores  (P2 • due:2025-12-15 • #uni)
[ ] ir al baño xd 
[✔] Comprar leche  (#casa)
[ ] Hacer ejercicio
[✔] Llamar a mamá
[ ] Preparar cena  (#casa)

# Eliminando tarea


# Completando otra tarea


# Mostrando todas las tareas finales

[ ] Estudiar compiladores  (P2 • due:2025-12-15 • #uni)
[ ] ir al baño xd 
[✔] Comprar leche  (#casa)
[✔] Preparar cena  (#casa)
[✔] Llamar a mamá


--- Tareas completadas ---
[✔] Comprar leche
[✔] Llamar a mamá
[✔] Preparar cena


In [30]:
# Ejecutar el programa demo y mostrar solo las tareas completadas
completed_tasks = run_and_get_completed(demo)
if completed_tasks:
    print("--- Tareas completadas ---")
    for task in completed_tasks:
        print(f"[✔] {task}")
else:
    print("(sin tareas completadas)")

--- Tareas completadas ---
[✔] Comprar leche
[✔] Llamar a mamá
[✔] Preparar cena


In [31]:
# Nuevo ejemplo
demo_nuevo = '''
task "Ir al gimnasio" tag "salud"
task "Leer un libro" priority 4
task "Hacer la compra" tag "casa"
task "Enviar email al profesor" priority 1 tag "uni"
task "Planificar viaje" tag "personal"

# Mostrando todas las tareas iniciales

show all

# Marcando tareas como completadas
done "Ir al gimnasio"
done "Leer un libro"

# Condición: Si "Ir al gimnasio" está completado, mostrar tareas completadas
if has "Ir al gimnasio" == done show done

# Condición: Si hay más de 1 tarea completada, mostrar tareas completadas
if count done > 1 show done

# Condición: Si "Enviar email al profesor" existe, completarla
if exists "Enviar email al profesor" done "Enviar email al profesor"

# Condición: Si "Planificar viaje" no está completada, mostrar tareas pendientes
if has "Planificar viaje" != done show pending

# Mostrando tareas pendientes finales
show pending

# Mostrando todas las tareas finales
show all
'''
print(run_program(demo_nuevo))






# Mostrando todas las tareas iniciales

[ ] Ir al gimnasio  (#salud)
[ ] Leer un libro  (P4)
[ ] Hacer la compra  (#casa)
[ ] Enviar email al profesor  (P1 • #uni)
[ ] Planificar viaje  (#personal)

# Marcando tareas como completadas



# Condición: Si "Ir al gimnasio" está completado, mostrar tareas completadas

Condición: has "Ir al gimnasio" == done
Resultado de la condición: Verdadera
[✔] Ir al gimnasio  (#salud)
[✔] Leer un libro  (P4)

# Condición: Si hay más de 1 tarea completada, mostrar tareas completadas

Condición: count done > 1
Resultado de la condición: Verdadera
[✔] Ir al gimnasio  (#salud)
[✔] Leer un libro  (P4)

# Condición: Si "Enviar email al profesor" existe, completarla

Condición: exists "Enviar email al profesor"
Resultado de la condición: Verdadera


# Condición: Si "Planificar viaje" no está completada, mostrar tareas pendientes

Condición: has "Planificar viaje" != done
Resultado de la condición: Verdadera
[ ] Hacer la compra  (#casa)
[ ] Planificar viaj

In [32]:
# Ejecutar el programa demo y mostrar solo las tareas completadas
completed_tasks = run_and_get_completed(demo_nuevo)
if completed_tasks:
    print("--- Tareas completadas ---")
    for task in completed_tasks:
        print(f"[✔] {task}")
else:
    print("(sin tareas completadas)")

--- Tareas completadas ---
[✔] Ir al gimnasio
[✔] Leer un libro
[✔] Enviar email al profesor


In [33]:

demo2 = '''
task "Hacer informe" priority 1 due "2025-11-30" tag "uni"
if has "Hacer informe" == done show done
done "Hacer informe"
if has "Hacer informe" == done show done
'''
print(run_program(demo2))



Condición: has "Hacer informe" == done
Resultado de la condición: Falsa


Condición: has "Hacer informe" == done
Resultado de la condición: Verdadera
[✔] Hacer informe  (P1 • due:2025-11-30 • #uni)


--- Tareas completadas ---
[✔] Hacer informe


In [34]:

def run_file(path: str):
    with open(path, "r", encoding="utf-8") as f:
        src = f.read()
    print(run_program(src))

# Ejemplo:
# with open("programa.todo","w",encoding="utf-8") as f:
#     f.write(demo)
# run_file("programa.todo")
